# Using Movielens 1M dataset

In [18]:
import pandas as pd

In [2]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']

In [9]:
users = pd.read_table('ml-1m/users.dat', sep='::', header = None, names = unames, engine= 'python')

In [11]:
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']

In [14]:
ratings= pd.read_table('ml-1m/ratings.dat', sep = '::', header = None, names = rnames, engine = 'python')

In [15]:
mnames = ['movie_id', 'title', 'genres']

In [17]:
movies = pd.read_table('ml-1m/movies.dat', sep = '::', header = None, names = mnames, engine = 'python', encoding = 'latin-1')

In [57]:
type(movies)

pandas.core.frame.DataFrame

In [19]:
users[:5]

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [20]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [21]:
movies[:5]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [22]:
# Analyzing the data spread across three tables is not a simple
# task; for example, suppose you wanted to compute mean ratings for a particular movie
# by sex and age. As you will see, this is much easier to do with all of the data merged
# together into a single table. Using pandas’s merge function, we first merge ratings with
# users then merging that result with the movies data. pandas infers which columns to
# use as the merge (or join) keys based on overlapping names:

In [23]:
data = pd.merge(pd.merge(users, ratings), movies)

In [24]:
data

,user_id,gender,age,occupation,zip,movie_id,rating,timestamp,title,genres
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...,...,...
1000204,5949,M,18,17,47901,2198,5,958846401,Modulations (1998),Documentary
1000205,5675,M,35,14,30030,2703,3,976029116,Broken Vessels (1998),Drama
1000206,5780,M,18,17,92886,2845,1,958153068,White Boys (1999),Drama
1000207,5851,F,18,20,55410,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western


In [26]:
mean_ratings = data.pivot_table('rating', index = 'title', columns = 'gender', aggfunc= 'mean')

In [27]:
mean_ratings[:5]

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024


In [28]:
ratings_by_title = data.groupby('title').size()

In [29]:
ratings_by_title[:5]

title
$1,000,000 Duck (1971)            37
'Night Mother (1986)              70
'Til There Was You (1997)         52
'burbs, The (1989)               303
...And Justice for All (1979)    199
dtype: int64

In [34]:
# filter down to movies that re-ceived at least 250 ratings (a completely arbitrary number);
active_titles = ratings_by_title.index[ratings_by_title > 250]

In [35]:
active_titles[:5]

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)',
       '12 Angry Men (1957)'],
      dtype='object', name='title')

In [37]:
# The index of titles receiving at least 250 ratings can then be used to select rows from mean_ratings above:

mean_ratings = mean_ratings.loc[active_titles]

In [38]:
mean_ratings

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
...,...,...
Young Guns (1988),3.371795,3.425620
Young Guns II (1990),2.934783,2.904025
Young Sherlock Holmes (1985),3.514706,3.363344


In [40]:
# To see the top films among female viewers, we can sort by the F column in descending order:

top_female_ratings = mean_ratings.sort_values(by = 'F', ascending=False)

In [41]:
top_female_ratings[:10]

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
"Shawshank Redemption, The (1994)",4.539075,4.560625
"Grand Day Out, A (1992)",4.537879,4.293255
To Kill a Mockingbird (1962),4.536667,4.372611
Creature Comforts (1990),4.513889,4.272277


# Measuring rating disagreement

In [43]:
# Suppose you wanted to find the movies that are most divisive between male and female
# viewers. One way is to add a column to mean_ratings containing the difference in
# means, then sort by that:

In [45]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

In [46]:
mean_ratings

gender,F,M,diff
title,,,
"'burbs, The (1989)",2.793478,2.962085,0.168607
10 Things I Hate About You (1999),3.646552,3.311966,-0.334586
101 Dalmatians (1961),3.791444,3.500000,-0.291444
101 Dalmatians (1996),3.240000,2.911215,-0.328785
12 Angry Men (1957),4.184397,4.328421,0.144024
...,...,...,...
Young Guns (1988),3.371795,3.425620,0.053825
Young Guns II (1990),2.934783,2.904025,-0.030758
Young Sherlock Holmes (1985),3.514706,3.363344,-0.151362


In [47]:
sorted_by_diff = mean_ratings.sort_values(by = 'diff')

In [48]:
sorted_by_diff

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777
...,...,...,...
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
"Longest Day, The (1962)",3.411765,4.031447,0.619682
Dumb & Dumber (1994),2.697987,3.336595,0.638608


In [49]:
# Reversing the order of the rows and again slicing off the top 15 rows, we get the movies
# preferred by men that women didn’t rate as highly:

sorted_by_diff[::-1][:15]

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,0.611985
"Hidden, The (1987)",3.137931,3.745098,0.607167
Rocky III (1982),2.361702,2.943503,0.581801
Caddyshack (1980),3.396135,3.969737,0.573602


In [50]:
# Suppose instead you wanted the movies that elicited the most disagreement among
# viewers, independent of gender. Disagreement can be measured by the variance or
# standard deviation of the ratings:

In [51]:
rating_std_by_title = data.groupby('title')['rating'].std()

In [52]:
rating_std_by_title

title
$1,000,000 Duck (1971)                        1.092563
'Night Mother (1986)                          1.118636
'Til There Was You (1997)                     1.020159
'burbs, The (1989)                            1.107760
...And Justice for All (1979)                 0.878110
                                                ...   
Zed & Two Noughts, A (1985)                   1.052794
Zero Effect (1998)                            1.042932
Zero Kelvin (Kjærlighetens kjøtere) (1995)    0.707107
Zeus and Roxanne (1997)                       1.122884
eXistenZ (1999)                               1.178568
Name: rating, Length: 3706, dtype: float64

In [53]:
# Filter down to active_titles
rating_std_by_title = rating_std_by_title.loc[active_titles]

In [54]:
rating_std_by_title

title
'burbs, The (1989)                   1.107760
10 Things I Hate About You (1999)    0.989815
101 Dalmatians (1961)                0.982103
101 Dalmatians (1996)                1.098717
12 Angry Men (1957)                  0.812731
                                       ...   
Young Guns (1988)                    1.017437
Young Guns II (1990)                 1.071959
Young Sherlock Holmes (1985)         0.891176
Zero Effect (1998)                   1.042932
eXistenZ (1999)                      1.178568
Name: rating, Length: 1214, dtype: float64

In [56]:
# Order the series in descending order

rating_std_by_title.sort_values(ascending = False)[:15]

title
Dumb & Dumber (1994)                           1.321333
Blair Witch Project, The (1999)                1.316368
Natural Born Killers (1994)                    1.307198
Tank Girl (1995)                               1.277695
Rocky Horror Picture Show, The (1975)          1.260177
Eyes Wide Shut (1999)                          1.259624
Evita (1996)                                   1.253631
Billy Madison (1995)                           1.249970
Fear and Loathing in Las Vegas (1998)          1.246408
Bicentennial Man (1999)                        1.245533
Hellraiser (1987)                              1.243046
Babe: Pig in the City (1998)                   1.239379
Wes Craven's New Nightmare (1994)              1.237630
South Park: Bigger, Longer and Uncut (1999)    1.235380
Deuce Bigalow: Male Gigolo (1999)              1.226337
Name: rating, dtype: float64